<a href="https://colab.research.google.com/github/raffeekk/ML/blob/main/%D0%A2%D0%B5%D0%BC%D0%B0%202/assignment02_habr_visual_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Домашнее задание № 2. <br> Визуальный анализ данных о публикациях на Хабрахабре</center>


In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка и знакомство с данными

Для работы вам понадобятся предобработанные данные нашего учебного конкурса на kaggle [«Прогноз популярности статьи на Хабре»](https://www.kaggle.com/c/howpop-habrahabr-favs). Скачайте [данные](https://drive.google.com/file/d/1nV2qV9otN3LnVSDqy95hvpJdb6aWtATk/view?usp=sharing) соревнования (данные были удалены с Kaggle ради организации последующего идентичного соревнования, так что тут ссылка на Google Drive).

In [3]:
# при необходимости поменяйте путь к данным
url = "https://raw.githubusercontent.com/raffeekk/ml/refs/heads/main/%D0%A2%D0%B5%D0%BC%D0%B0%202/howpop_train.csv"
df = pd.read_csv(url)

In [ ]:
df.shape

In [ ]:
df.head(3).T

Избавимся сразу от переменных, названия которых заканчиваются на `_lognorm` (нужны для соревнования на Kaggle). Выберем их с помощью `filter()` и удалим `drop`-ом:

In [ ]:
df.drop(
    filter(lambda c: c.endswith("_lognorm"), df.columns),
    axis=1,  # axis = 1: столбцы
    inplace=True,
)  # избавляет от необходимости сохранять датасет

In [ ]:
df.describe().T

In [ ]:
df.describe(include=["object", "bool"]).T  # бинарные и категориальные переменные

In [ ]:
# настройка внешнего вида графиков в seaborn
sns.set_style("dark")
sns.set_palette("RdBu")
sns.set_context(
    "notebook", font_scale=1.5, rc={"figure.figsize": (15, 5), "axes.titlesize": 18}
)

Столбец **`published`** (время публикации) содержит строки. Чтобы мы могли работать с этими данными как с датой/временем публикации, приведём их к типу `datetime`:

In [ ]:
print(df.published.dtype)
df["published"] = pd.to_datetime(df.published, yearfirst=True)
print(df.published.dtype)

Создадим несколько столбцов на основе данных о времени публикации:

In [ ]:
df["year"] = [d.year for d in df.published]
df["month"] = [d.month for d in df.published]

df["dayofweek"] = [d.isoweekday() for d in df.published]
df["hour"] = [d.hour for d in df.published]

-----
Теперь Ваша очередь. В каждом пункте предлагается построить картинку и с ее помощью ответить на вопрос в [форме](https://docs.google.com/forms/d/e/1FAIpQLSf3b5OG8zX_nLQBQ-t20c6M5Auz-VUL-yxj8Fm9_o_XWDBTrg/viewform?c=0&w=1). Конечно, можно попытаться ответить на все вопросы только с Pandas, без картинок, но мы советуем Вам потренироваться строить (красивые) визуализации.   

## 1\. В каком месяце (и какого года) было больше всего публикаций?

* март 2016
* март 2015
* апрель 2015
* апрель 2016

In [ ]:
month_year_counts = df.groupby(["year", "month"]).size().reset_index(name="counts")
max_month = month_year_counts.loc[month_year_counts.counts.idxmax()]
print(f"Больше всего публикаций было в {max_month['month']} {max_month['year']} года.")

## 2\. Проанализируйте публикации в месяце из предыдущего вопроса

Выберите один или несколько вариантов:

* Один или несколько дней сильно выделяются из общей картины
* На хабре _всегда_ больше статей, чем на гиктаймсе
* По субботам на гиктаймс и на хабрахабр публикуют примерно одинаковое число статей

Подсказки: постройте график зависимости числа публикаций от дня; используйте параметр `hue`; не заморачивайтесь сильно с ответами и не ищите скрытого смысла :)

In [ ]:
m15 = df[(df['year'] == 2015) & (df['month'] == 3)]
publications_per_day = m15.groupby('dayofweek')['published'].count()
plt.figure(figsize=(10, 5))
publications_per_day.plot(kind='bar')
plt.xlabel('Дни недели')
plt.ylabel('Количество публикаций')
plt.title('Количество публикаций по дням (март 2015 года)')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

## 3\. Когда лучше всего публиковать статью?

* Больше всего просмотров набирают статьи, опубликованные в 12 часов дня
* У опубликованных в 10 утра постов больше всего комментариев
* Больше всего просмотров набирают статьи, опубликованные в 6 часов утра
* Максимальное число комментариев на гиктаймсе набрала статья, опубликованная в 9 часов вечера
* На хабре дневные статьи комментируют чаще, чем вечерние

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df.groupby("hour").views.mean().reset_index(),
    x="hour", y="views", label="Среднее количество просмотров"
)
sns.lineplot(
    data=df.groupby("hour").comments.mean().reset_index(),
    x="hour", y="comments", label="Среднее количество комментариев"
)
plt.title("Средние просмотры и комментарии по времени публикации")
plt.xlabel("Час публикации")
plt.ylabel("Среднее значение")
plt.legend()
plt.show()

## 4\. Кого из топ-20 авторов чаще всего минусуют?

* @Mordatyj
* @Mithgol
* @alizar
* @ilya42

In [ ]:
top_20_authors = df["author"].value_counts().head(20).index
author_minus = df[df.author.isin(top_20_authors)].groupby("author")["votes_minus"].sum()
most_minus = author_minus.idxmax()
print(f"Автор с наибольшим числом минусов: @{most_minus}")

## 5\. Сравните субботы и понедельники

Правда ли, что по субботам авторы пишут в основном днём, а по понедельникам — в основном вечером?

In [ ]:
sat_mon = df[df.dayofweek.isin([1, 6])]  # 1 = Понедельник, 6 = Суббота
plt.figure(figsize=(12, 6))
sns.histplot(
    data=sat_mon, x="hour", hue="dayofweek", multiple="stack",
    palette="Set1", bins=24
)
plt.title("Распределение публикаций по часам для понедельника и субботы")
plt.xlabel("Час публикации")
plt.ylabel("Количество публикаций")
plt.legend(["Понедельник", "Суббота"], title="День недели")
plt.show()